# Text-based Information Retrieval

## Assignment PART II
### Using wordembedding
We can use the semantic similarity of wordembeddings, such as GloVe and Word2Vec, to obtain better results.
In this part of the exercise, we will the addition analogy (similar to Part I of this assignment) to rank the given documents.


In [1]:
# Loading modules
import os, re
import pandas as pd
from numpy import dot, sum
#from gensim import matutils, models, corpora, similarities
import gensim

# Set up logger that logs (works in jupyter 3!) in console and outputs in file
'''
import logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='part_II_logs.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.DEBUG)
'''

"\nimport logging\nlogger = logging.getLogger()\nfhandler = logging.FileHandler(filename='part_II_logs.log', mode='a')\nformatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')\nfhandler.setFormatter(formatter)\nlogger.addHandler(fhandler)\nlogger.setLevel(logging.DEBUG)\n"

#### Images to wordvectors

We will use the similarity of wordmodels such as Word2Vec and GloVe to make vectors of each image. These vectors will look like 
>s = w1 + w2 + ... + wn

> With s = the image vector and {w1 .. wn} the words for each image


In [2]:
# Load in a stopword list from
# http://www.lextek.com/manuals/onix/stopwords2.html
stopwords = []
with open('data/stopwordlist.txt', 'r') as f:
    lines = ''.join(f.readlines())
    stopwords = [x for x in lines.split('\n')[2:]]


In [3]:
# Clean input because the wordmodels can not contain every possible combination words and signs
def clean_input(text, stopwords):
    # lowecase and remove linebreaks
    text = text.lower().rstrip()
    # Remove stopwords
    text = ' '.join([word for word in text.split() if word not in stopwords])
    # Remove punctuation
    text = re.sub('[!@#$:;%&?,_\.\'\`\"\\\/\(\)\[\]]', '', text)
    text = re.sub('[\-]+', '-', text)
    # Remove sole numbers, dashes or extra spaces
    text = re.sub('[\s][\-]+[\s]', '', text)
    text = re.sub('[0-9]+', '', text)
    text = re.sub('[\s]+', ' ', text)
    # British to American English - at this moment still hardcoded due to lack of library
    text = text.replace('grey', 'gray')
    text = text.replace('colour', 'color')
    text = text.replace('tyre', 'tire')
    text = text.replace('centre', 'center')
    text = text.replace('theatre', 'theater')
    text = text.replace('jewellery','jewelry')
    text = text.replace('aeroplane', 'plane')
    text = text.replace('harbour', 'harbor')
    text = text.replace('moustache','mustache')
    text = text.replace(' axe', ' hatchet')
    text = text.replace('armour', 'armor')
    text = text.replace('stylised', 'stylized')
    text = text.replace('organise', 'organize')
    text = text.replace('plough', 'plow')
    text = text.replace('neighbourhood', 'neighborhood')
    text = text.replace('vapour', 'vapor')
    # some manual fixes of lemmatizing
    text = text.replace('watersid ', 'waterside ')
    text = text.replace('figur ', 'figure ')
    text = text.replace(' graz ', ' graze ')
    text = ' '.join([word for word in text.split() if word not in stopwords])
    return text


In [4]:
# Text file parser
# Returns a dictionary with imageid - text in lowercase without stopwords or punctuation
def text_file_parser(filename, stopwords):
    corpus = dict()
    #corpus = pd.DataFrame(columns=('id', 'imageid', 'vec'))
    with open(filename) as f:
        # next(f) # skip first line with the headings
        for doc in f:
            # Split on spaces
            doc_parts = doc.split(" ", 1)
            # If first part is the ID (needed for the queries file)
            if(len(doc_parts[0]) < 6):
                doc_parts = doc.split(" ", 2)
                doc_parts.pop(0)
            # Clean the caption text (remove puctuation etc)
            doc_parts[1] = clean_input(doc_parts[1], stopwords)
            # add the array (or row) to an array
            corpus[len(corpus) + 1] = doc_parts        
    # Transform to dataframe
    df = pd.DataFrame.from_dict(corpus, orient='index')
    df = df.reset_index()
    df.columns = ['index', 'img_id', 'caption']
    return df

In [5]:
# images file to docs dict
print('Parsing documents')
training_docs = text_file_parser('data/target_collection_parsed.txt', stopwords)
docs = []
for text in training_docs['caption']:
    docs.append(text.split())
dictionary = gensim.corpora.Dictionary(docs)
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(doc) for doc in docs]   
# generate LDA model
lda = gensim.models.ldamodel.LdaModel(corpus, num_topics=300, id2word = dictionary, passes=10, alpha="auto")
queries = text_file_parser('data/queries_val_parsed.txt', stopwords)

# Preview
queries

Parsing documents


index            img_id  \
0        1  CbnA_e6AW6U7Ycoa   
1        2  XBZPztvt67qkMUdI   
2        3  PaqtOaYmQmXkqW2i   
3        4  IPcFtNL-7EQ6Z0yu   
4        5  IMAD0sq2Fz7HpSgX   
5        6  -gqRDDfPZTGlCfJa   
6        7  xsrYb57vl4qiMLDG   
7        8  BCjxgJlQ3TD5T8ST   
8        9  LGxwsl9CtRQ8wW3Y   
9       10  9LtOvyiygFYoxU8S   
10      11  8usTLD-Wg5EHCShk   
11      12  kH59MJp3nWyFfFB2   
12      13  tluPF-CA6dN6LACF   
13      14  HKXYBXXObkt_yi7s   
14      15  mBJjuuB0ukfKmnRH   
15      16  cqpTaVCZZe5OpuIk   
16      17  Anry0qU5NFes6Twh   
17      18  ByjuNEqsLcbUk5OH   
18      19  5dBiwoDEpY6gWRek   
19      20  ZFdHuWXCk662UDAW   
20      21  BqoajT5F2iTYrBVY   
21      22  6e3OOBOz0U2yUPw2   
22      23  fSA2edOk84knSsCd   
23      24  b1BJi-1w-Rx0TTYR   
24      25  K4WT0f0TDFUF1oss   
25      26  RDoxm_sjkO6p-jfU   
26      27  4U7OnXftrQUXbSwZ   
27      28  8Yc1fvVrq4t0Jg0g   
28      29  _-Y9MEwXW1byM5Ia   
29      30  O5cAfwjvftQzafZ3   
..     ...               ...   
970    971  Z1TKwXyWzPHIFyqo   
971    972  WsHd9WhpLTVCLUlq   
972    973  JKPdw6y2FTJxsh1K   
973    974  5-I1KW3scIc4pbsZ   
974    975  v4My8_bo8z3Xwqxc   
975    976  FhwdAJ46w3nyp8TX   
976    977  VH-qvLkwpmbFCwOK   
977    978  hawj8QzTA-Eil_rD   
978    979  eQ4lbzbmh7k60mlp   
979    980  Sh0zz4xg3Y4m3qPv   
980    981  -dp39Cdz3XJstMoS   
981    982  HipIEk7-Itnp9PcB   
982    983  8-FHUXbxYGlRM5LL   
983    984  BetXL-_TeL926a8t   
984    985  UqXOPOVpFVKDFJyn   
985    986  TojsF8C4bNb0UKvQ   
986    987  -q-lecPJgKXcZnOI   
987    988  56qZnrVS3CzBG9Rs   
988    989  fynWgHfhy5sjydX6   
989    990  yW8dpEWfc3wwO3J6   
990    991  BCjxgJlQ3TD5T8ST   
991    992  mMPiMRMAOsllXJX1   
992    993  -PNOjxOjVGR7tmk7   
993    994  DYvDr2CFE77pikTb   
994    995  AtSw6SmdQ2-PdnWL   
995    996  onL79JCBLbPO5WE6   
996    997  8k3j8rDXPor6INeN   
997    998  XxYyaPP_Yno4x935   
998    999  tliWJPOil1cylMd5   
999   1000  6xf0y9zkKrEWqMK5   

                                               caption  
0              people group holding award pose picture  
1    man white shirt sit table cut meat plate front...  
2                       woman red dress posing hatchet  
3                  soccer play stand soccer ball front  
4                             white yellow train track  
5                              view tall building city  
6                                hand pick flower vine  
7                              picture army ready sail  
8                             brick roof house picture  
9                                man clean mess street  
10           group kid play playground accompany adult  
11      woman black shirt jean pant picture red carpet  
12                                boat sail body water  
13            crowd people stadium watch baseball game  
14                           men drag large snake tail  
15                           group people street beach  
16                woman sit bottom bed neat hotel room  
17                    man wear glove mask destroy wall  
18             brick entrance building tree background  
19     woman bright red lipstick sing large microphone  
20      picture people climb snow-covered mountain day  
21                                 men woman stand row  
22                          young girl rid bike street  
23                           bike stand lock bike rack  
24                            dog wrong eye teeth miss  
25                          man young woman pose photo  
26                                 rocky ruin blue sky  
27                            men play basketball dunk  
28                          lady black smile lean wall  
29                           woman wear glass hold pen  
..                                                 ...  
970                             large bowl cook oyster  
971                  white airplane runway green grass  
972                 man woman wear fancy attire camera  
973  blonde woman wear wh

#### Check similarity

In [6]:
tfidf = gensim.models.TfidfModel(corpus)
length = len(dictionary)
sim_index = gensim.similarities.Similarity('./tmp/tst',corpus, num_features=length)
print('querying')

'''
# Calculate similarity
#training_docs["sim"] = 0.0
queries['recall'] = 0.0
queries['precision'] = 0.0
for i,r in queries.iterrows():
    
    # Calculate similarity per vector document
    vec_bow = dictionary.doc2bow(r['caption'].split())
    vec_lda = lda[vec_bow]
    sims = index[vec_lda]
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    sims = [sim[0] for sim in sims[:1000]]
    
    # Recall
    total_to_find = float(len(training_docs[training_docs.img_id == r['img_id']]))
    correct = 0
    for sim in sims:
        if training_docs[training_docs.index == sim].values[0][1] == r['img_id']:
            correct += 1
    
    amount_found = correct
    recall = float(amount_found/total_to_find)
    queries.set_value(i, 'recall', recall)
    
    # Precision
    precision = float(amount_found/float(1000))
    queries.set_value(i, 'precision', precision)

print('Finished querying')
'''

# Create outputfile if not already exists
#if not os.path.exists(output_file):
#    open(output_file, 'w').close()

# Create a results dataframe that will contain the MAP and average recall for all run queries
results = pd.DataFrame(index=range(0,1000), columns=['map', 'recall'])
results['map'] = 0.0
results['recall'] = 0.0

# Calculate similarity
print ("Calculating precision and recall")
for i,r in queries.iterrows():
    print ("Processing query", i) # Print the current query number
    
    # Calculate similarity per vector document
    vec_bow = dictionary.doc2bow(r['caption'].split())
    vec_lda = lda[vec_bow]
    sims = sim_index[vec_lda]
    
    # order by similarity and get top 1000 results
    sims = sorted(enumerate(sims), key=lambda item: -item[1])
    sims = [sim[0] for sim in sims[:1000]]
    
    # Get total amount of images that should be found
    total_to_find = float(len(training_docs[training_docs.img_id == r['img_id']]))
    total_found = 0.0
    current_avg_precision = 0.0
    
    # Iterare over top 1000 found documents
    for index, sim in enumerate(sims):
        if training_docs[training_docs.index == sim].values[0][1] == r['img_id']:
            total_found += 1.0
            
        # Append values to dataframe for precision-recall curve
        current_avg_precision = float(current_avg_precision) + float(total_found/float(index + 1.0))
        curr_results = results.loc[index]
        results.set_value(index, "recall", float(curr_results['recall']) + float(total_found/total_to_find))
        results.set_value(index, "map", float(curr_results['map']) + float(current_avg_precision/float(index + 1.0)))
        
        # Append to the output file
        #with open(output_file,'a') as f:
        #    # Output in the file is structured as:
        #    # query_id 0 sent_id 0 similarity_score 0
        #    f.write(str(r['index'])+' 0 '+str(row['index'])+' 0 '+str(row['sim'])+" 0\n")
    
# Reduce results rows to averages
results['recall'] = results.apply((lambda row: float(float(row['recall'])/1000.0)), axis=1)
results['map'] = results.apply((lambda row: float(float(row['map'])/1000.0)), axis=1)

querying
Calculating precision and recall
Processing query 0
Processing query 1
Processing query 2
Processing query 3
Processing query 4
Processing query 5
Processing query 6
Processing query 7
Processing query 8
Processing query 9
Processing query 10
Processing query 11
Processing query 12
Processing query 13
Processing query 14
Processing query 15
Processing query 16
Processing query 17
Processing query 18
Processing query 19
Processing query 20
Processing query 21
Processing query 22
Processing query 23
Processing query 24
Processing query 25
Processing query 26
Processing query 27
Processing query 28
Processing query 29
Processing query 30
Processing query 31
Processing query 32
Processing query 33
Processing query 34
Processing query 35
Processing query 36
Processing query 37
Processing query 38
Processing query 39
Processing query 40
Processing query 41
Processing query 42
Processing query 43
Processing query 44
Processing query 45
Processing query 46
Processing query 47
Processi

#### Results

In [7]:
print ("Displaying results:")

print ("--")
print ("MAP for best 10:", results.loc[9]['map'])
print ("MAP for best 100:", results.loc[99]['map'])
print ("MAP for best 250:", results.loc[249]['map'])
print ("MAP for best 500:", results.loc[499]['map'])
print ("MAP for best 1000:", results.loc[999]['map'])

print ("AVG recall for best 10", results.loc[9]['recall'])
print ("AVG recall for best 100", results.loc[99]['recall'])
print ("AVG recall for best 250", results.loc[249]['recall'])
print ("AVG recall for best 500", results.loc[499]['recall'])
print ("AVG recall for best 1000", results.loc[999]['recall'])
print ("--")

# print results
print (results)

Displaying results:
--
MAP for best 10: 0.000874404761905
MAP for best 100: 0.00127835271594
MAP for best 250: 0.00117450441248
MAP for best 500: 0.00112939321511
MAP for best 1000: 0.00106944473165
AVG recall for best 10 0.00115810086811
AVG recall for best 100 0.0110988061106
AVG recall for best 250 0.0228248133427
AVG recall for best 500 0.0451026322661
AVG recall for best 1000 0.0788785285645
--
          map    recall
0    0.000000  0.000000
1    0.000000  0.000000
2    0.000111  0.000022
3    0.000333  0.000312
4    0.000467  0.000412
5    0.000583  0.000464
6    0.000684  0.000676
7    0.000739  0.000676
8    0.000805  0.000869
9    0.000874  0.001158
10   0.000927  0.001301
11   0.000968  0.001412
12   0.001000  0.001483
13   0.001031  0.001775
14   0.001060  0.001947
15   0.001091  0.002313
16   0.001117  0.002413
17   0.001141  0.002538
18   0.001159  0.002538
19   0.001171  0.002538
20   0.001185  0.002991
21   0.001200  0.003300
22   0.001210  0.003300
23   0.001217  0.0033

In [8]:
%matplotlib inline
import matplotlib.pyplot as plt

# Sort by recall
#df = df.sort_values(by=['recall'], ascending=[1])

# Show and save chart
qx = results.plot(x='recall', y='map')
#qx.set_xlim([0,1])
#qx.set_ylim([0,1])
fig = qx.get_figure()
fig.savefig('results/part2_lda_precision-recall.png')

### Results from calculations on server
For lowering the workload on our computers, we let a server do the calculations and write the results to a csv file.


In [ ]:
# Read in results file
results = pd.DataFrame.from_csv('results/results.csv')

# averages
print "--"
print "AVG recall", results.recall.mean()
print "AVG precision:",  results.precision.mean()
print "--"

# Preview dataframe
results